In [1]:
import torch

In [2]:
# Get data
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [3]:
with open('data/dr_seuss.txt', 'r', encoding='utf-8') as in_file:
    text = in_file.read()

print("Length of dataset in characters:", len(text))

Length of dataset in characters: 96052


In [3]:
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [4]:
# Get all unique characters in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("Chars:", ''.join(chars))
print("N unique chars:", vocab_size)

Chars: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
N unique chars: 65


In [6]:
# Define character-level tokenizers
c_to_i = {ch: i for i, ch in enumerate(chars)}
i_to_c = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [c_to_i[c] for c in s]
decode = lambda l: ''.join([i_to_c[i] for i in l])

print(encode('hello there'))
print(decode(encode('hello there')))

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
hello there


In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# Train / test split
train_pct = 0.9
n_train = int(data.shape[0] * train_pct)
train_data = data[:n_train]
val_data = data[n_train:]

train_data.shape, val_data.shape

(torch.Size([1003854]), torch.Size([111540]))

In [9]:
# Example of (x, y) pairs in next-token prediction
block_size = 8

print(f"block: {train_data[:block_size + 1]}")
x = train_data[:block_size]
y = train_data[1: block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target is: {target}")

block: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
when input is tensor([18]), the target is: 47
when input is tensor([18, 47]), the target is: 56
when input is tensor([18, 47, 56]), the target is: 57
when input is tensor([18, 47, 56, 57]), the target is: 58
when input is tensor([18, 47, 56, 57, 58]), the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is: 58


In [10]:
def get_batch(data, batch_size, block_size):
    # Randomly sample offsets in the data to construct mini batches with
    offsets = torch.randint(len(data) - block_size, (batch_size,))
    # Construct batch tensors
    x = torch.stack([data[i: i+block_size] for i in offsets])
    y = torch.stack([data[i+1: i+block_size+1] for i in offsets])
    return x, y

# Show example
torch.manual_seed(1337)
batch_size = 4
block_size = 8

xb, yb = get_batch(train_data, batch_size, block_size)
print('inputs:', xb.shape)
print(xb)
print('targets:', yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context}, the target is: {target}")

inputs: torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when input is tensor([24]), the target is: 43
when input is tensor([24, 43]), the target is: 58
when input is tensor([24, 43, 58]), the target is: 5
when input is tensor([24, 43, 58,  5]), the target is: 57
when input is tensor([24, 43, 58,  5, 57]), the target is: 1
when input is tensor([24, 43, 58,  5, 57,  1]), the target is: 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]), the target is: 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), the target is: 39
when input is tensor([44]), the target is: 53
when input is tensor([44, 53]), the target is: 56
when input is tensor([44, 53, 56]